In [ ]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

mlflow.set_tracking_uri("http://127.0.0.1:8081")

with mlflow.start_run() as run:
    X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    params = {"max_depth": 2, "random_state": 42}
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)

    # Infer the model signature
    y_pred = model.predict(X_test)
    signature = infer_signature(X_test, y_pred)

    # Log parameters and metrics using the MLflow APIs
    mlflow.log_params(params)
    mlflow.log_metrics({"mse": mean_squared_error(y_test, y_pred)})

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="sk-learn-random-forest-reg-model",
    )


In [ ]:
# Note: You can get the value from MLFLow UI

result = mlflow.register_model(
    "UPDATE_HERE", "sk-learn-random-forest-reg"
)
# Example:
# result = mlflow.register_model(
#     "runs:/1eb52d7d4a354856a52dd8598462d87d/sklearn-model", "sk-learn-random-forest-reg"
# )


In [ ]:
from mlflow import MlflowClient

client = MlflowClient()
client.create_registered_model("sk-learn-random-forest-reg-model")

# Note: You will get and error since model "sk-learn-random-forest-reg-model" already exists.

In [ ]:
client = MlflowClient()
result = client.create_model_version(
    name="sk-learn-random-forest-reg-model",
    source="UPDATE_HERE",
    run_id="UPDATE_HERE",
)
# Note: You can get the above values from MLFLow UI

## Example:
# result = client.create_model_version(
#     name="sk-learn-random-forest-reg-model",
#     source="mlflow-artifacts:/0/1eb52d7d4a354856a52dd8598462d87d/artifacts/sklearn-model",
#     run_id="1eb52d7d4a354856a52dd8598462d87d",
# )

In [ ]:
import mlflow.pyfunc

model_name = "sk-learn-random-forest-reg-model"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model.predict(X_test)


In [ ]:
# Archive models version 1 from Production into Archived
client = MlflowClient()
client.transition_model_version_stage(
    name="sk-learn-random-forest-reg-model", version=1, stage="Archived"
)
